### Importing Libraries

In [7]:
import tkinter as tk  
from tkinter import ttk
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askopenfilename
import tkinter as tk

import pandas as pd
import numpy as np
from tkinter.filedialog import askopenfilename
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.layers import BatchNormalization, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

### Class for import CSV and target column

In [32]:
#Import csv funtion
def import_csv_data():
    file_path = askopenfilename()
    csv_path=file_path
    csv_path=csv_path.split("/")
    csv_path=csv_path[len(csv_path)-1]
    v.set(csv_path)
    a.df1=pd.read_csv(file_path)
    
    import_data_status="Initialize"
    if csv_path=="":
        import_data_status="No Data"
    else:
        import_data_status="Done"
    import_data_text.set(import_data_status)
    

#identify target variable funtion
def target_variable():
    tar_val_status='Initialize'
    tar_val=target_var.get()
    print(tar_val_status)
    for col in a.df1.columns:
        str=''
        str+=col
    if col==tar_val:
        tar_val_status="Target variable Found"
    else:
        tar_val_status="Target variable not Found"
    print(tar_val_status)
    tar_val_text.set(tar_val_status) 


### Class for Neural network Regressor train and test

In [9]:
   #neural network regressor function
def reg_preprocess():
    global X,y
    X=a.df1.drop("Signal_Strength",axis=1)
    y=a.df1["Signal_Strength"]
    scaler=StandardScaler()
    X=scaler.fit_transform(X)
    return X,y

def reg_train_command():
    global reg_model,X_val,y_val
    reg_preprocess()
    #Train-Test split
    X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.30,random_state=40)
    print(X_train)
    reg_model=tf.keras.models.Sequential()
    reg_model.add(BatchNormalization())
    reg_model.add(Dense(3,input_shape=(11,),activation='relu'))
    reg_model.add(BatchNormalization())
    reg_model.add(Dense(3,input_shape=(11,),activation='relu'))
    reg_model.add(BatchNormalization())
    reg_model.add(Dense(1,input_shape=(11,),activation='relu'))
    reg_model.add(Dense(1))

    sgd = optimizers.SGD(lr=1e-2, decay=1e-6, momentum=0.9)
    reg_model.compile(optimizer=sgd, loss='mse')
    reg_fit=reg_model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=100, verbose=1)
    
    reg_status="initialize"
    if reg_model=="":
        reg_status="Model not Trained"
    else:
        reg_status="Model Trained"
    reg_train_value_text.set(reg_status)
    print(X_val)
    return reg_model,X_val,y_val

def reg_test_command():  
    ypred = reg_model.predict(X_val).flatten()
    print(X_val)
    pred_data={"Actual Value":np.array(y_val),"Predicted":ypred}
    print(pred_data)
    ypred_data=pd.DataFrame(pred_data)
    
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.csv')
    ypred_data.to_csv (export_file_path, index = False, header=True)
    reg_test_status="initialize"
    if export_file_path=="":
        reg_test_status="Not Saved"
    else:
        reg_test_status="Saved to Disk"
    reg_value_text.set(reg_test_status)

    print(ypred)


### Class for Neural network Classifier train and test

In [10]:
#neural network classifier function
def cla_preprocess():
    global X_train_c,X_val_c,y_train_c,y_val_c,y_c
    reg_preprocess()
    encoder = LabelEncoder()
    y_c=encoder.fit_transform(y)
    X_train_c,X_val_c,y_train_c,y_val_c=train_test_split(X,y_c,test_size=0.30,random_state=40)
    y_train_c=tf.keras.utils.to_categorical(y_train_c,num_classes=6,dtype=float)
    y_val_c=tf.keras.utils.to_categorical(y_val_c,num_classes=6)
    return X,y_c,X_train_c,X_val_c,y_train_c,y_val_c

def clas_train_command():
    global c_model,X_val_c,y_val_c
    cla_preprocess()  
    #Changing Target column to categorical
    c_model=tf.keras.models.Sequential()
    c_model.add(BatchNormalization())
    c_model.add(Dense(3,input_shape=(11,),activation='relu'))
    c_model.add(BatchNormalization())
    c_model.add(Dense(3,input_shape=(11,),activation='relu'))
    c_model.add(Dense(6, activation='softmax'))
    c_model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
    c_model.fit(X_train_c, y_train_c, validation_data=(X_val_c,y_val_c), epochs=100, verbose=1)
    
    class_status="initialize"
    if c_model=="":
        class_status="Model not Trained"
    else:
        class_status="Model Trained"
    class_train_value_text.set(class_status)
    print(X_val_c)
    return c_model,X_val_c,y_val_c,y_c
    
def clas_test_command():
    
    ypred_c  = c_model.predict_classes(X_val_c).flatten()
    y_val_trans=tf.math.argmax(y_val_c, axis=1)
    #ypred =pd.DataFrame(np.array(ypred_c) ,index=["3","4","5","6","7","8"])
    print(ypred_c)
    pred_data={"Actual Value":np.array(y_val_trans),"Predicted":np.array(ypred_c)}
    print(pred_data)
    ypred_data=pd.DataFrame(pred_data)
    
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.csv')
    ypred_data.to_csv (export_file_path, index = False, header=True)
    
    clas_test_status="initialize"
    
    if export_file_path=="":
        clas_test_status="Not Saved"
    else:
        clas_test_status="Saved to Disk"
    clas_value_text.set(clas_test_status)

### Neural network GUI

In [31]:
class app:
    def __init__(self):
        pass
    global df1
    df1=pd.DataFrame(index=range(1,11))

a=app()
root = tk.Tk()

#variable initialization
v=tk.StringVar()
tar_val_text=tk.StringVar()
impor_var_text=tk.StringVar()
reg_value_text=tk.StringVar()
clas_value_text=tk.StringVar()
reg_train_value_text=tk.StringVar()
class_train_value_text=tk.StringVar()
import_data_text=tk.StringVar()

#File upload and target variable upload
tk.Label(root, text='Step1: File Name').grid(row=0, column=0,sticky=W)
import_data_button=tk.Button(root, text='Import Data',command=import_csv_data).grid(row=0, column=2)
entry = tk.Entry(root,textvariable= v).grid(row=0, column=1)
import_data_entry=tk.Entry(root,textvariable= import_data_text).grid(row=0, column=3)

target_var = tk.Entry(root)
target_var.insert(END,"Signal_Strength")
target_var.grid(row=2, column=1,sticky=W)
target_data_button=tk.Button(root, text='ImportTarget',command=target_variable).grid(row=2, column=2)
target_variable_status=tk.Entry(root,textvariable=tar_val_text).grid(row=2, column=3)

#Neural Network Regressor
tk.Label(root, text='Step3: Neural network Regressor').grid(row=2, column=0,sticky=W)
tk.Label(root, text='Regressor').grid(row=3, column=0)
reg_train=tk.Button(root, text='Train',command=reg_train_command).grid(row=3, column=1)
reg_entry_status = tk.Entry(root,textvariable= reg_train_value_text).grid(row=3, column=2)
reg_test=tk.Button(root, text='Test',command=reg_test_command).grid(row=4, column=1)
reg_entry_test_status = tk.Entry(root,textvariable= reg_value_text).grid(row=4, column=2)

#Neural Network Classifier
tk.Label(root, text='Step3: Neural network Classifier').grid(row=5, column=0,sticky=W)
tk.Label(root, text='Classifier').grid(row=6, column=0)
clas_train=tk.Button(root, text='Train',command=clas_train_command).grid(row=6, column=1)
clas_entry_status = tk.Entry(root,textvariable= class_train_value_text).grid(row=6, column=2)
clas_test=tk.Button(root, text='Test',command=clas_test_command).grid(row=7, column=1)
clas_entry_test_status = tk.Entry(root,textvariable= clas_value_text).grid(row=7, column=2)


#Close Button
tk.Button(root, text='Close',command=root.destroy).grid(row=8, column=2) 

root.mainloop()
    